In [ ]:
##Creates one file which contains only the important substructure for each type of fingerprints.
##This kind of output for machine learning algorithms gave scores around 5% worse than using CDK fingerprints
##The cut-off of percentage difference between the first and the second category can be changed in the line "if difference > X"

In [25]:
import numpy as np

In [26]:
##Creates one dictionary for each file
categoriesFile = open ("categoriesProcessed.csv")
categoriesDictionary={}
for line in categoriesFile:
    line=line.strip("\n")
    line=line.split(",")
    categoriesDictionary[line[0]]=line[1]
categoriesFile.close()
fingerprintsFile = open ("allFingerprintsCombinedWithoutDuplicates.csv")##The substructures that are always the same don't matter
fingerprintsDictionary={}
for line in fingerprintsFile:
    line = line.strip("\n")
    line = line.split(",")
    fingerprintsDictionary[line[0]]=line[1]
fingerprintsFile.close()

In [27]:
##Joins the two dictionaries
##This is not the ideal way of doing it but dictionaries don't accept numpy arrays as a key. The two lists are in the same order.
listOfFingerprints=[]
listOfCategories=[]
for clusterName in fingerprintsDictionary.keys():
    listOfFingerprints.append(list(fingerprintsDictionary[clusterName]))
    listOfCategories.append(categoriesDictionary[clusterName])

In [28]:
##Let's calculate the indexes of each category and the total number of products in each category
alkaloidIndexes=[]
nrpIndexes=[]
nucleosideIndexes=[]
otherIndexes=[]
polyketideIndexes=[]
rippIndexes=[]
saccharideIndexes=[]
terpeneIndexes=[]
for i in range(0, 1031):
    if listOfCategories[i] == 'Alkaloid':
        alkaloidIndexes.append(i)
    elif listOfCategories[i] == 'NRP':
        nrpIndexes.append(i)
    elif listOfCategories[i] == 'Nucleoside':
        nucleosideIndexes.append(i)
    elif listOfCategories[i] == 'Other':
        otherIndexes.append(i)
    elif listOfCategories[i] == 'Polyketide':
        polyketideIndexes.append(i)
    elif listOfCategories[i] == 'RiPP':
        rippIndexes.append(i)
    elif listOfCategories[i] == 'Saccharide':
        saccharideIndexes.append(i)
    elif listOfCategories[i] == 'Terpene':
        terpeneIndexes.append(i)
numberOfAlkaloid=len(alkaloidIndexes)
numberOfNrp=len(nrpIndexes)
numberOfNucleoside=len(nucleosideIndexes)
numberOfOther=len(otherIndexes)
numberOfPolyketide=len(polyketideIndexes)
numberOfRipp=len(rippIndexes)
numberOfSaccharide=len(saccharideIndexes)
numberOfTerpene=len(terpeneIndexes)

In [29]:
##NUCLEOSIDE WILL NOT BE TAKEN INTO ACCOUNT AS THERE IS ONLY ONE COMPOUND AND THE PERC WILL BE 0 OR 100 ALL THE TIME
##First, let's identify which substructures are important
importantSubstructuresList=[]
importantSubstructuresIndexList=[]
for i in range (0, 9365):
    alkaloidPerc=(sum(listOfFingerprints[j][i]=='1' for j in alkaloidIndexes))/numberOfAlkaloid*100
    nrpPerc=(sum(listOfFingerprints[j][i]=='1' for j in nrpIndexes))/numberOfNrp*100
    otherPerc=(sum(listOfFingerprints[j][i]=='1' for j in otherIndexes))/numberOfOther*100
    polyketidePerc=(sum(listOfFingerprints[j][i]=='1' for j in polyketideIndexes))/numberOfPolyketide*100
    rippPerc=(sum(listOfFingerprints[j][i]=='1' for j in rippIndexes))/numberOfRipp*100
    saccharidePerc=(sum(listOfFingerprints[j][i]=='1' for j in saccharideIndexes))/numberOfSaccharide*100
    terpenePerc=(sum(listOfFingerprints[j][i]=='1' for j in terpeneIndexes))/numberOfTerpene*100
    percList = []
    percList.append (alkaloidPerc)
    percList.append (nrpPerc)
    percList.append (otherPerc)
    percList.append (polyketidePerc)
    percList.append (rippPerc)
    percList.append (saccharidePerc)
    percList.append (terpenePerc)
    maximum = max(percList)
    maximumPos = percList.index(max(percList))
    percList.sort()
    secondMax=percList[-2]
    difference = maximum-secondMax
    if difference > 30:
        #print (str(difference) + "  "+str(i) + " " )
        importantSubstructuresList.append(i)
        importantSubstructuresIndexList.append(maximumPos)

In [30]:
##Now, let's identify for which category each substructure is important
for i in range (0, len(importantSubstructuresList)):
    category=''
    if importantSubstructuresIndexList[i] == 0:
        category='Alkaloids'
    elif importantSubstructuresIndexList[i] == 1:
        category='NRPs'
    elif importantSubstructuresIndexList[i] == 2:
        category='Others'
    elif importantSubstructuresIndexList[i] == 3:
        category='Polyketides'
    elif importantSubstructuresIndexList[i] == 4:
        category='RiPPs'
    elif importantSubstructuresIndexList[i] == 5:
        category='Saccharides'
    elif importantSubstructuresIndexList[i] == 6:
        category='Terpenes'
    #print ("The substructure number %d is clearly differentiated for %s" % (importantSubstructuresList[i], category))

In [31]:
print (len (importantSubstructuresList))

451


In [32]:
##Creates a new file and writes only the most important substructures with more than 35% difference between the first and the second category
outputFile = open ("combinedImportantSubstructures.csv","w")
inputFile = open ("allFingerprintsCombinedWithoutDuplicates.csv")
for line in inputFile:
    line = line.strip("\n")
    outputFile.write(line[0:11])##Name Of The cluster plus ","
    i=0
    for character in line[11:]:
        if i in importantSubstructuresList:
            outputFile.write(character)
        i += 1
    outputFile.write("\n")
inputFile.close()
outputFile.close()